In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
train_pre = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_mean = train_pre.data.float().mean() / 255 
train_std = train_pre.data.float().std() / 255

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(train_mean,train_std)])

batch_size = 512

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.fc1(x)
        return nn.functional.softmax(x)
    
net = ConvNet() 

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

num_epochs = 5
print_every = 100

for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

/var/folders/07/65yfr7y95sd5nhnx51849p580000gn/T/ipykernel_32541/471972347.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(x)


[1,   100] loss: 2.059
[2,   100] loss: 1.657
[3,   100] loss: 1.570
[4,   100] loss: 1.547
[5,   100] loss: 1.532
[6,   100] loss: 1.523
[7,   100] loss: 1.516
[8,   100] loss: 1.510
[9,   100] loss: 1.505
[10,   100] loss: 1.501
[11,   100] loss: 1.498
[12,   100] loss: 1.495
[13,   100] loss: 1.493
[14,   100] loss: 1.491
[15,   100] loss: 1.489
[16,   100] loss: 1.488
[17,   100] loss: 1.486
[18,   100] loss: 1.485
[19,   100] loss: 1.484
[20,   100] loss: 1.484
[21,   100] loss: 1.483
[22,   100] loss: 1.481
[23,   100] loss: 1.482
[24,   100] loss: 1.480
[25,   100] loss: 1.480
[26,   100] loss: 1.479
[27,   100] loss: 1.479
[28,   100] loss: 1.478
[29,   100] loss: 1.477
[30,   100] loss: 1.477
[31,   100] loss: 1.477
[32,   100] loss: 1.476
[33,   100] loss: 1.476
[34,   100] loss: 1.476
[35,   100] loss: 1.476
[36,   100] loss: 1.475
[37,   100] loss: 1.475
[38,   100] loss: 1.474
[39,   100] loss: 1.474
[40,   100] loss: 1.474
[41,   100] loss: 1.474
[42,   100] loss: 1.473
[

In [5]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(test_accuracy)

/var/folders/07/65yfr7y95sd5nhnx51849p580000gn/T/ipykernel_32541/471972347.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(x)


98.74
